# Agent 的并行和串行


## 一、引言

> We always overestimate the change that will occur in the next two years and underestimate the change that will occur in the next ten. -- Bill Gates

一年前，和同事讨论大语言模型 (LLM) 何时能取代程序员。我的判断是最快 3 年，最晚 5 年，程序员这个职业就不存在了。2024 年是大模型产品化的一年，Sora, [GR](https://arxiv.org/abs/2402.17152), GPT-4o 相继涌现。那段时间，我疯狂使用 GPT。惊叹从搜索引擎到 GPT，人类搜索效率又上了一个台阶。每天都能在 𝕏 上刷到关于 LLM 最新的点子，大模型可以应用的地方似乎无穷无尽，一遍又一遍刷新我对 LLM 能力边界的认知。

但是到了 2025 年，我的看法发生了一些变化。我没有那么乐观了。我现在觉得 AGI 很遥远，也许还要很多个 5 年才能实现。

为什么我的看法变了？

### 1. 尚未到来的涌现

2024 年的 LLM 缺少工程。我的预期是补全工程，让大模型具备自我调用能力，或许可以实现自举，从而带来一波智能涌现。如今工程能力 (MCP, Agent) 已经齐备，预想中的涌现却没有到来。

这预示着 AGI 没有捷径，必须要底层模型 AGI 才能 AGI。而仅靠 LLM 范式，能否 scaling 到 AGI 的程度，实属未知。

### 2. 万能拟合函数

我常说神经网络不是万能拟合函数，论据是它连预处理都不会。你说堂堂万能拟合函数，具有拟合一切分布的能力。怎么会需要人类把预处理好的数据送到嘴边，才能吃得下去？若有不知死活的人类，把未经预处理的生数据喂下去，马上就掉 GAUC 给你看。

所以你看，深度学习时代，模型离了人真不行。大模型时代呢？还真有可能发生改变。

大模型具有世界知识，如果大模型的能力强大到可以做数据预处理、数据合成、数据评估和数据筛选，并且通用到能以少量编程驱动这些模块，那么自动化的深度学习或许真的可以实现。考虑到大部分算法工程师本质是做样本和特征，那么可以一举优化掉至少一半的算法工程师 ^_^

### 3. 小米加步枪

Agent 的意思是，大模型的底层能力不够，就用工程的方式去拼去凑。

```mermaid
graph LR
    A[感知] --> B[规划]
    B --> C[控制]
    C -.-> D[反思]
    D -.-> E[记忆]
```

Agent 的任务处理框架大多遵循如下模式：

- **感知**：一个任务过来，无论是 **被动接受** 任务信息，还是 **主动探索** 与任务有关的线索，这都属于“感知”。
- **规划**：有了对任务的认知，通过拆解任务，规划出一条或多条实现路径
- **控制**：有了实现路径，在路径上的每一个点，通过 MCP 调度资源执行任务；如果当前点遇到的问题依旧复杂，通过子 Agent 继续拆解任务
- **反思**：如果有多条实现路径，对多条路径的结果进行评估；如果只有一条实现路径，则对当前结果进行确认和校验
- **记忆**：收集本次任务信息，提炼要点，存入知识库。下次执行相似任务时，作为先验知识

一个 LLM 搞不定，那就组织多个 LLMs 来解决。这有点艰苦奋斗的意思。没有大炮没关系，就用小米加步枪的方式实现。

### 4. Agent 为何强大

Agent 一方面可以通过 MCP 调度计算和存储资源，包括但不限于：

- FastAPI（可接入多种中台能力）
- PostgreSQL
- Redis

另一方面可以通过子 Agent 进一步拆解任务，形成一种可生长的树状调度结构。这显然比单个 LLM 强大得多！

### 5. 没有 AGI，我们能做什么

但是 Agent 依然受到底层模型能力的制约。一个表现是：**特定领域的 Agent 容易，通用 Agent 很难**。一切都跟 AGI 有关。一旦通用性不足，就陷入硬编码 (hard coding) 的无底洞去了。

当前盘面下，我们可以做 Agent 能做到，但传统编程做不到的事！

**1）智能路由**

Agent 是一个超级 **调度器**。在复杂情况下，可以自主决定调度何种资源。

**2）“无限”分治**

人的精力有限，没有办法将 **任务拆解** 做到极致。而服务器的精力近乎无限，可以不厌其烦地拆解问题。

## 二、技术选型

我的技术选型如下：

- **LLM Model**: Qwen3-0.6B-FP8
- **Inference Engine**: vLLM
- **Agent**: Qwen Agent
- **RAG**: LangChain
- **Vector Engine**: Chroma

通过简单的 [预研项目](https://github.com/luochang212/mcp-project?tab=readme-ov-file#预研计划)，我跑完了 OpenAI Agents 和 Qwen Agent 的示例代码。OpenAI 在概念上更胜一筹，它定义了很多重要概念，比如 交接 (Handoffs)、护栏 (Guardrails) 等。这方面 Qwen 确实是简化了一些，用户实现同等功能需要不小的开发量。

但我还是选择了 Qwen Agent。没有选择 OpenAI Agent 是因为 OpenAI 太不 Open 了。它主要支持 `/v1/responses` 接口，缺少对开源界流行的 `/v1/chat/completions` 接口的支持。这意味着由 vLLM 部署的服务无法享受 OpenAI Agents 的部分新特性。并且像 Hosted tools 这类内置工具，只有 OpenAI API 的用户才能使用。

最重要的是，我在运行 OpenAI 代码的时候，时常受阻。因为 OpenAI Agents 将付费客户作为一等公民，开源用户的体验则要差上许多，在文档和代码方面皆是如此。尽管我没有选择 OpenAI Agents，但依然建议学习它。它的概念、架构、设计思路确有独到之处，值得吸收借鉴。

# 两种把 Agent 用起来的思路

<!-- > 本来想用“范式”的，奈何范式这个词实在太大了，还是用“思路”比较妥当 -->

> 技术的发展大抵是 “拿着钉子找锤子” 和 “拿着锤子找钉子” 交替前进的过程，现在轮到拿着锤子找钉子了。

## 引言

把 Agent 放进实际业务前，先问自己：把代码换成 Agent，收益在哪里？

如果 Agent 实现的东西，编程也能实现。这种情况下用 Agent，无异于为赋新词强说愁。要知道 Agent 出场自带 debuff。由于大模型 (LLM) 的参数量庞大，它的加入自然会降低系统的吞吐量 (QPS)；而且 LLM 的 next token 有随机性，它还会让系统的稳定性下降。所以 Agent 轻易不能出场，必须要有不得不用的理由。

**Agent 的价值在编程的能力圈之外**。

如你所见，Agent 出场是有成本 (Marginal Cost) 的。如果它只能做到传统编程能做到的东西，我们倾向于不使用它。因此，一定要在传统编程范式“做不到”的地方，寻找 Agent 的价值。

我们需要对比一下，找到 Agent 的优势：

**1）文本处理**

LLM 擅长处理文本，文本任务天然适合 Native LLM 的处理思路。而且由于文本模态的抽象程度高，简直是包罗万象，相比其他模态更适合作为 **世界模型**，这意味着它能够吸收这个世界的通用知识。

**2）柔性计算**

传统编程范式下，编程依赖于可计算，而可计算依赖于优良的定义。但现实世界中，不是所有东西都可以被优良定义的。如果可以优良定义，写 `if-else` 块就好啦。相比于硬编码 (hard coding)，柔性计算才是困难的地方。

```mermaid
graph LR
    A[决策树] --> B[机器学习]
    B --> C[深度学习]
```




**3）“无限”分治**

感知 规划 控制 执行 评估 反思




# Agent

## 引言

**Agent 的价值在编程的能力圈之外**。

在把 Agent 放进实际业务之前，首先问自己：从代码换成 Agents 实现，收益在哪里？

因为现有编程范式足够强大，用 Agent 能实现的东西，很可能直接编程也能实现。这种情况下用 Agent，无异于为赋新词强说愁。降低了系统的吞吐性能不说，由于 LLM 的输出具有随机性，也会导致业务的稳定性下降。因此，一定要在传统编程范式“够不到”的地方，寻找 Agent 的价值。

LLM 在哪些方面强于传统编程范式呢？我想了一下，至少有以下三个方面：

**1）文本处理**

Agent 擅长文本处理，尤其是多任务串行的文本处理。由于 LLM 本身对语言模态的亲和力，这类任务往往越是用 Native LLM 的思路处理，效果越好。这里还有一个超级场景，就是 **论文速递**。

论文速递就是把新发表的论文收集起来，提炼核心内容。然后筛选出有价值的论文，推送到群聊，以帮助自己和朋友们快速感知本领域最新的技术方向。这种多步骤的文本处理任务，正是 Agent 最擅长的。

**2）组织能力**

人非常善于“分治思维”。不同于程序的 `if else`，人类分治思维的优越性在于可以处理模糊的映射关系。传统编程范式下，分治依赖于可计算，可计算依赖于优良的定义。但是现实世界中，不是所有东西都能被清晰地定义。一旦遇到定义模糊的情况，hard coding 便无法解决。虽然后续发展出 决策树、深度学习 等更加柔性的计算范式，依然无法与人类的处理复杂问题地能力相比拟。如今的 LLM 已经是最接近人类处理能力的技术。

分治的难点在于拆解问题的准度。比如，大厂有相对完善的中台能力，员工很多时间只是充当调度器的角色。比如，老板派了个 A 活，就去中台调用 $\alpha$ 服务；派 B 活，就去调用 $\beta$ 服务。看似简单，但是活儿可能是非标的，可能需要对任务做预处理，

人在工作中处理的一类问题叫做“有条件触发”。举个例子，大厂有相对完善的中台能力，员工很多时候只是充当调度器的角色，老板派了个 A 活，我们就去中台调用 $\alpha$ 服务；派 B 活，我们就去调用 $\beta$ 服务。这中间为什么不能自动化呢？因为有两个问题不好解决：

- 语言模态和机器代码的交互
- x 任务调 y 服务是一种松散映射



加上最近逐渐火起来的 MCP Server，LLM 拥有了广泛的使用工具的能力。

**3）探索能力**

```mermaid
graph LR
    A[任务] --> B[调度者]
    B --> C[mcp1]
    B --> D[mcp2]
    B --> E[mcp3]
```

In [ ]:
调度


保研名额确认


- system prompt 规则
- input 带有信息
- llm 理解规则
- mcp 缺少的信息调用运算资源获取
- 总结/判断




In [ ]:
探索

计算密集任务；只要满足一个条件就拦截

- 计算触发条件
    - 是否有大量关联用户
    - 是否有疑似聚集行为
    - case：检查reason字段

- 对威胁等级排序，产出排序，输出探索计划

- 按排序顺序调度资源

- 如果命中，调度检查程度



论文速递













```mermaid
graph LR
    A[由reason触发] --> B(遍历线索，生成探索计划)
    B --> C[探索计划1]
    C -->|失败| D[探索计划2]
    D -->|失败| E[探索计划3]
    E -->|失败| G[通过]

    C -->|成功| F{检查程序}
    D -->|成功| F
    E -->|成功| F

    subgraph 检查逻辑
    F -->|失败| G
    F -->|成功| H[拦截]
    end
```

In [ ]:
ip 时序相似

## 4. 一点遐想

Agent 技术带来许多遐想。我个人比较期待 Agent 实现预处理的自动化。

我们说，神经网络不是万能拟合函数，理由是它必须做预处理。堂堂万能拟合函数，肯定要有拟合一切分布的能力。深度学习差就差在这里。它必须做预处理，否则无法充分学习数据里的信息。若有不知死活的工程师，把未经预处理的数据注入模型，GAUC 马上掉给你看。可见预处理对深度学习之重要。

要知道，在深度模型卷无可卷的当下，与样本和特征相关的工作，正占用工程师越来越多的时间。Agent 拥有世界知识，并且有强大的泛化性，很有潜力完成数据清洗、处理、合成、评估等一系列工作。如果 Agent 能将工程师从预处理中解放出来，无疑是大好事一件。